In [248]:
import http.client
import json
import pandas as pd
from pandas.io.json import json_normalize
import requests
import config

In [262]:
url1 = f"https://api.sportmonks.com/v3/football/fixtures/date/2022-09-06?api_token={config.api_key}&include=scores;participants"
url2 = f"https://api.sportmonks.com/v3/football/fixtures/date/2022-09-03?api_token={config.api_key}&include=scores;participants"

response2 = requests.get(url2)
data2 = response2.json()
print('data' in data2)

response1 = requests.get(url1)
data1 = response1.json()
print('data' in data1)



True
False


In [249]:
url = f"https://api.sportmonks.com/v3/football/fixtures/date/2022-09-03?api_token={config.api_key}&include=scores;participants"

response = requests.get(url)
data = response.json()
df = pd.DataFrame(data['data'])\
       .drop(columns=['id', 'sport_id', 'stage_id', 'league_id', 'season_id', 'group_id', 'aggregate_id', 'round_id', 'state_id', 'venue_id', 'leg','details','length','placeholder','has_odds','starting_at_timestamp'])

In [250]:
df

,name,starting_at,result_info,scores,participants
0,Celtic vs Rangers,2022-09-03 11:30:00,Celtic won after full-time.,"[{'id': 11316768, 'fixture_id': 18535517, 'typ...","[{'id': 53, 'sport_id': 1, 'country_id': 1161,..."
1,Hibernian vs Kilmarnock,2022-09-03 14:00:00,Hibernian won after full-time.,"[{'id': 11316774, 'fixture_id': 18535518, 'typ...","[{'id': 66, 'sport_id': 1, 'country_id': 1161,..."
2,Livingston vs Hearts,2022-09-03 14:00:00,Livingston won after full-time.,"[{'id': 11316780, 'fixture_id': 18535519, 'typ...","[{'id': 258, 'sport_id': 1, 'country_id': 1161..."
3,Motherwell vs Dundee United,2022-09-03 14:00:00,Game ended in draw.,"[{'id': 11316786, 'fixture_id': 18535520, 'typ...","[{'id': 309, 'sport_id': 1, 'country_id': 1161..."
4,Ross County vs Aberdeen,2022-09-03 14:00:00,Game ended in draw.,"[{'id': 11316792, 'fixture_id': 18535521, 'typ...","[{'id': 273, 'sport_id': 1, 'country_id': 1161..."
5,St. Johnstone vs St. Mirren,2022-09-03 14:00:00,St. Johnstone won after full-time.,"[{'id': 11316798, 'fixture_id': 18535522, 'typ...","[{'id': 734, 'sport_id': 1, 'country_id': 1161..."


In [251]:
scores = pd.json_normalize(df.scores, sep='_').rename(columns={4:'home_score', 5:'away_score'}).drop(columns=[0,1,2,3])
home_scores = pd.json_normalize(scores.home_score, sep='_')\
                .rename(columns={'score_goals':'total_home_goals'})\
                .drop(columns=['score_participant', 'description', 'type_id', 'fixture_id', 'id', 'participant_id'])

away_scores = pd.json_normalize(scores.away_score, sep='_')\
                .rename(columns={'score_goals':'total_away_goals'})\
                .drop(columns=['score_participant', 'description', 'type_id', 'fixture_id', 'id', 'participant_id'])



In [252]:

participants = pd.json_normalize(df.participants, sep='_').rename(columns={0:'home', 1:'away'})

away = pd.json_normalize(participants.away, sep='_')\
         .rename(columns={'id':'away_id', 'name':'away_name', 'image_path':'away_image_path', 'founded':'away_founded', 'short_code': 'away_short_code'})\
         .drop(columns=['sport_id', 'type', 'placeholder', 'meta_location', 'meta_winner', 'meta_position', 'gender', 'last_played_at', 'venue_id'])

home = pd.json_normalize(participants.home, sep='_')\
         .rename(columns={'id':'home_id', 'name':'home_name', 'image_path':'home_image_path', 'founded':'home_founded', 'short_code': 'home_short_code'})\
         .drop(columns=['sport_id', 'type', 'placeholder', 'meta_location', 'meta_winner', 'meta_position', 'gender', 'last_played_at', 'venue_id'])

df = pd.concat([df, home, away, home_scores, away_scores], axis=1)

In [253]:
df



,name,starting_at,result_info,scores,participants,home_id,country_id,home_name,home_short_code,home_image_path,home_founded,away_id,country_id,away_name,away_short_code,away_image_path,away_founded,total_home_goals,total_away_goals
0,Celtic vs Rangers,2022-09-03 11:30:00,Celtic won after full-time.,"[{'id': 11316768, 'fixture_id': 18535517, 'typ...","[{'id': 53, 'sport_id': 1, 'country_id': 1161,...",53,1161,Celtic,CEL,https://cdn.sportmonks.com/images/soccer/teams...,1888,62,1161,Rangers,RAN,https://cdn.sportmonks.com/images/soccer/teams...,1873,4,0
1,Hibernian vs Kilmarnock,2022-09-03 14:00:00,Hibernian won after full-time.,"[{'id': 11316774, 'fixture_id': 18535518, 'typ...","[{'id': 66, 'sport_id': 1, 'country_id': 1161,...",66,1161,Hibernian,HIB,https://cdn.sportmonks.com/images/soccer/teams...,1875,180,1161,Kilmarnock,KIL,https://cdn.sportmonks.com/images/soccer/teams...,1869,1,0
2,Livingston vs Hearts,2022-09-03 14:00:00,Livingston won after full-time.,"[{'id': 11316780, 'fixture_id': 18535519, 'typ...","[{'id': 258, 'sport_id': 1, 'country_id': 1161...",258,1161,Livingston,LIV,https://cdn.sportmonks.com/images/soccer/teams...,1943,314,1161,Hearts,None,https://cdn.sportmonks.com/images/soccer/teams...,1874,1,0
3,Motherwell vs Dundee United,2022-09-03 14:00:00,Game ended in draw.,"[{'id': 11316786, 'fixture_id': 18535520, 'typ...","[{'id': 309, 'sport_id': 1, 'country_id': 1161...",309,1161,Motherwell,MOT,https://cdn.sportmonks.com/images/soccer/teams...,1886,282,1161,Dundee United,DUD,https://cdn.sportmonks.com/images/soccer/teams...,1909,0,0
4,Ross County vs Aberdeen,2022-09-03 14:00:00,Game ended in draw.,"[{'id': 11316792, 'fixture_id': 18535521, 'typ...","[{'id': 273, 'sport_id': 1, 'country_id': 1161...",273,1161,Aberdeen,ABE,https://cdn.sportmonks.com/images/soccer/teams...,1903,246,1161,Ross County,RSC,https://cdn.sportmonks.com/images/soccer/teams...,1929,1,1
5,St. Johnstone vs St. Mirren,2022-09-03 14:00:00,St. Johnstone won after full-time.,"[{'id': 11316798, 'fixture_id': 18535522, 'typ...","[{'id': 734, 'sport_id': 1, 'country_id': 1161...",734,1161,St. Johnstone,STJ,https://cdn.sportmonks.com/images/soccer/teams...,1885,496,1161,St. Mirren,STM,https://cdn.sportmonks.com/images/soccer/teams...,1877,3,0
